In [1]:
import pandas as pd
import numpy as np


In [173]:
df=pd.read_excel("GlasgowJobs.xlsx")

In [95]:
df.head(5)

,Unnamed: 0,Title,Company,Location,Link,Status,salary,Description
0,0,Assistant Store Manager,DFS Furniture Stores,Glasgow,https://uk.indeed.com/viewjob?jk=a477287423f067d9,PostedPosted 2 days ago,"£65,000 a year","They coach, motivate, and inspire their collea..."
1,1,Assistant Venue Manager,Vega,Glasgow G2,https://uk.indeed.com/viewjob?jk=e7bfca102a9f49a8,PostedPosted 15 days ago,"£26,000 a year",We are looking for someone who loves to coach ...
2,2,Assistant manager,Gamba,Glasgow,https://uk.indeed.com/viewjob?jk=c9c5dda56c4d4d58,PostedPosted 2 days ago,"£27,000 a year",You would have responsability for assistaning ...
3,3,Assistant Manager,Savoy Nightclub,Glasgow G2,https://uk.indeed.com/viewjob?jk=cff3fe3553c380e5,EmployerActive 3 days ago,"£21,000 - £23,000 a year",Day to day running of the business.
4,4,Assistant Manager,HY Hospitality Group,Glasgow G2,https://uk.indeed.com/viewjob?jk=130b741f7def4cab,PostedToday,"From £24,000 a year",45 hour contracted working week.


In [96]:
df.shape

(180, 8)

In [174]:
df=df.sort_values("Status").drop_duplicates("Company",keep="last")
df.shape

(57, 8)

In [175]:
#We extract each of the groups in in the status to later build a timer
employer=df["Status"].str.extract("(Employer)(\d{1,2})")
posted=df["Status"].str.extract(".(Posted)\s(\d{1,2})*")
posted2=df["Status"].str.extract("(Posted)(\d{1,2})")
active=df["Status"].str.extract(".(Active)\s(\d{1,2})")
df["Status"]=df["Status"].str.replace("PostedJust posted","PostedToday")
df["Status"]=df["Status"].str.replace("PostedToday","Posted Today")
postedToday=df["Status"].str.extract("(Posted)\s(Today)")


#Time to fill all the dataframe 
posted=posted.fillna(active)
posted=posted.fillna(employer)
posted=posted.fillna(posted2)
posted=posted.fillna(postedToday)
#wecould do this replace in regex, but i though was going to be easy do it later on
posted[1]=posted[1].replace("Today","0")

#Now we change to int to can use the timedelta
posted[1]=posted[1].astype(int)

#And we apply a lambda to each number to transform it in a date
posted[1]=posted[1].apply(lambda date: (pd.to_datetime("today")-pd.Timedelta(days=date)).strftime("%d/%m/%Y") )

df[["Status","Date"]]=posted


In [176]:
df.head()

,Unnamed: 0,Title,Company,Location,Link,Status,salary,Description,Date
81,81,Deputy Manager (Domiciliary Care),J C Michael Groups Ltd,London,https://uk.indeed.com/viewjob?jk=4be69e395c037b5d,Employer,"£30,000 a year",Experience of building positive working relati...,30/07/2022
168,168,Assistant Manager,German Doner Kebab Glasgow,Glasgow G2,https://uk.indeed.com/viewjob?jk=ae608027da7e83fd,Active,"£25,000 a year",Not Found,27/08/2022
13,13,Assistant Restaurant Manager,Celinos,Glasgow G31,https://uk.indeed.com/viewjob?jk=ec1e6b66f3f78dce,Active,£10 - £11 an hour,The successful candidate will have experience ...,27/08/2022
175,175,Deputy General Manager,Garvie and Co,Glasgow G62,https://uk.indeed.com/viewjob?jk=37bb6b351a26fee2,Active,"Up to £28,000 a year",Not Found,27/08/2022
146,146,Assistant Conference & Banqueting Manager,Glasgow Grosvenor Hotel,Glasgow G12,https://uk.indeed.com/viewjob?jk=18c4a1645c4473cf,Active,"From £25,000 a year","A range of benefits are included, staff meals,...",26/08/2022


Now we use regex to clean the data and create the new columns that we gonna use.

In [177]:
#we divide the colum salary in 2
df[["Starter_Salary","Top_Salary"]]=df["salary"].str.extract(r"£(\d{2},*\d*)?(?:.*)?£(\d*,*\d*)")
df["Top_Salary"]=df["Top_Salary"].fillna(df["salary"].str.extract(r"£([0-9,]*)")[0])
                                

df.drop("salary",axis=1,inplace=True)
df["Starter_Salary"].fillna(df["Top_Salary"],inplace=True) 

#we use lambda to clean the "," from the numbers and convert the type to float so we can use it
df[["Starter_Salary","Top_Salary"]]=df[["Starter_Salary","Top_Salary"]].apply(lambda x: x.str.replace(",","")).astype(float)

# we clean the posible spaces from the database
df["Location"]=df["Location"].str.strip()

# And we extract the  city and the area from the location
city=df["Location"].str.extract(r"^(?:Hybrid)[^A-Z]*([A-Z][a-z]*)|([A-Z][a-z]*)")
city[0].fillna(city[1],inplace=True)
df["City"]=city[0]

df["Area"]=df["Location"].str.extract(r"([A-Z]\d{1,2})")
df.drop("Location", axis=1, inplace=True)

#Clean The title
df["Title"]=df["Title"].str.split().apply(lambda title: [word.capitalize() for word in title]).str.join(" ")


In [178]:
# We tidy up a bit the board
df.sort_values("Starter_Salary",ascending=False,inplace=True)
df.set_index(df.columns[0],inplace=True)
df.index.name=""

df.head(10)

,Title,Company,Link,Status,Description,Date,Starter_Salary,Top_Salary,City,Area
,,,,,,,,,,
0,Assistant Store Manager,DFS Furniture Stores,https://uk.indeed.com/viewjob?jk=a477287423f067d9,Posted,"They coach, motivate, and inspire their collea...",27/08/2022,65000.0,65000.0,Glasgow,NaN
114,Assistant Payroll Manager,The Portfolio Group,https://uk.indeed.com/viewjob?jk=fcbd88831e49431d,Posted,Confident with manual payroll/use of spreadshe...,27/08/2022,55000.0,60000.0,London,NaN
66,Assistant Project Manager,Sopra Steria,https://uk.indeed.com/viewjob?jk=9c05f843a646d8f5,Posted,You will either be working in support of a Pro...,29/08/2022,40000.0,45000.0,Remote,NaN
80,Assistant Block Manager,ONE PROPERTY RECRUITMENT LTD,https://uk.indeed.com/viewjob?jk=f64711c9f3e55fe2,Posted,Responding to leasehold enquiries in a custome...,27/08/2022,35000.0,35000.0,Remote,NaN
30,Associate Delivery Manager (delivery Team Supp...,HM Revenue and Customs,https://uk.indeed.com/viewjob?jk=d86c8eda56135639,Posted,Not Found,26/08/2022,34404.0,36985.0,Glasgow,NaN
15,Assistant Manager,Spark of Genius,https://uk.indeed.com/viewjob?jk=5168d9ef7b4fd08a,Posted,Not Found,30/07/2022,32670.0,35220.0,Mauchline,A5
18,Assistant Project Manager,SSE,https://uk.indeed.com/viewjob?jk=fc5f2b2e3c70632e,Posted,Not Found,13/08/2022,32600.0,48800.0,Remote,NaN
14,Assistant Project Manager,SSE PLC,https://uk.indeed.com/viewjob?jk=fc5f2b2e3c70632e,Posted,Lead the co-ordination of and provide input to...,13/08/2022,32600.0,48800.0,Remote,NaN
109,Sehsct_deputy Endoscopy Manager (band 6) Perma...,Health and Social Care Trust,https://uk.indeed.com/viewjob?jk=c6fe8cca6ffd54da,Posted,Not Found,29/08/2022,32000.0,39000.0,Belfast,NaN


## Visualization

We gonna start to go throw some visualization basic from the data, just to see if everything check, before start ploting grafics

In [179]:

df.loc[36] # we can use loc or Iloc to looc for diferent jobs offerts




Title              Assistant Manager/operations Manager - Part-time
Company                                                      Wickes
Link              https://uk.indeed.com/viewjob?jk=dd0c2775a709a078
Status                                                       Posted
Description                                               Not Found
Date                                                     19/08/2022
Starter_Salary                                                  NaN
Top_Salary                                                      NaN
City                                                        Glasgow
Area                                                            NaN
Name: 36, dtype: object

**This block gonna help us to visualizate what kind of titles are better pay and also which are the most offer.**
- As you can see the data is not full clean, as if we go very hard with regex, when we change to another pages or another kind     of jobs, this dont gonna capturate all the information, but to do future studies we could rename some of them, as always gonna   be just few of them to need rename.



In [180]:
#Jpaymore stands for Jobs that pay more
Jpaymore=df.groupby("Title").max("Starter_Salary")
Jpaymore["Count"]=df.groupby("Title")["Title"].count()
Jpaymore.reset_index(inplace=True)
#creating a serie to extract the tittle to reduce the variance
payserie=Jpaymore["Title"].str.extract(r"(^[A-z][a-z]+\s[A-za-z\s&]+)|(.*)")
payserie[0].fillna(payserie[1],inplace=True)
Jpaymore["Title"]=payserie[0]
Jpaymore.sort_values(by="Starter_Salary",ascending=False,inplace=True)

Jpaymore

,Title,Starter_Salary,Top_Salary,Count
18,Assistant Store Manager,65000.0,65000.0,2
13,Assistant Payroll Manager,55000.0,60000.0,1
14,Assistant Project Manager,40000.0,48800.0,4
1,Assistant Block Manager,35000.0,35000.0,1
23,Associate Delivery Manager,34404.0,36985.0,1
7,Assistant Manager,32670.0,35220.0,18
33,Sehsct_deputy Endoscopy Manager (band 6) Perma...,32000.0,39000.0,1
30,Id2022-3554 - Assistant Finance Manager,31542.0,36129.0,1
4,Assistant General Manager,31000.0,35000.0,2
5,Assistant General Manager,30000.0,30000.0,1


In [181]:
df[df["Title"].str.contains("Assistant Manager")&(df["Starter_Salary"]>=25000)]

,Title,Company,Link,Status,Description,Date,Starter_Salary,Top_Salary,City,Area
,,,,,,,,,,
15,Assistant Manager,Spark of Genius,https://uk.indeed.com/viewjob?jk=5168d9ef7b4fd08a,Posted,Not Found,30/07/2022,32670.0,35220.0,Mauchline,A5
23,Assistant Manager,Farmhouse Inns,https://uk.indeed.com/viewjob?jk=042757a3e664db03,Posted,Driving Licence,28/08/2022,28000.0,28000.0,Glasgow,G33
87,Assistant Manager,Marstons Plc - Queen of the Loch,https://uk.indeed.com/viewjob?jk=c6f1a33451b2a7f8,Active,Not Found,26/08/2022,28000.0,28000.0,Balloch,G83
118,Assistant Manager,Clockwise,https://uk.indeed.com/viewjob?jk=ca32562adac7ad12,Active,IT literacy the ability to use Windows based t...,25/08/2022,27000.0,27000.0,Glasgow,G2
2,Assistant Manager,Gamba,https://uk.indeed.com/viewjob?jk=c9c5dda56c4d4d58,Posted,You would have responsability for assistaning ...,27/08/2022,27000.0,27000.0,Glasgow,NaN
124,Assistant Manager - Glasgow,Las Iguanas,https://uk.indeed.com/viewjob?jk=aab1b85969fe15d1,Posted,25% Friends and Family Discount.,26/08/2022,26500.0,28000.0,Glasgow,G1
29,Assistant Manager,Restaurant Growth,https://uk.indeed.com/viewjob?jk=afba0d9bcaca750d,Posted,Not Found,26/08/2022,26000.0,26000.0,Glasgow,G64
8,Assistant Manager - Flannels,Flannels,https://uk.indeed.com/viewjob?jk=6cb507636987a6da,Posted,"Trend aware, fashion conscious and well presen...",30/07/2022,25250.0,35000.0,Glasgow,G34
64,Assistant Manager,Balloch House,https://uk.indeed.com/viewjob?jk=bd082e6561f75a25,Active,View all Balloch House jobs - Alexandria jobs,22/08/2022,25000.0,28000.0,Alexandria,G83


In [182]:
# For now we gonna have all store in separate excels to check later as we are not in the point yet to build something consistent.

df.to_excel("GlasgowCleanData.xlsx")
df.to_excel("Groups.xlsx")

